<a href="https://colab.research.google.com/github/J0hnV1ct0r/Formacao_ML_Avancada_Alura/blob/main/WordEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados

In [1]:
import pandas as pd

In [2]:
artigo_treino = pd.read_csv("/content/drive/MyDrive/alura/treino.csv")

In [3]:
artigo_teste = pd.read_csv("/content/drive/MyDrive/alura/teste.csv")

In [4]:
len(artigo_treino)

90000

In [5]:
len(artigo_teste)

20513

# One-hot-enconding

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
texto = ["tenha um bom dia", "tenha um péssimo dia", "tenha um ótimo dia","tenha um dia ruim"]
vetorizador = CountVectorizer()
vetorizador.fit(texto)

CountVectorizer()

In [8]:
print(vetorizador.vocabulary_)

{'tenha': 4, 'um': 5, 'bom': 0, 'dia': 1, 'péssimo': 2, 'ótimo': 6, 'ruim': 3}


In [9]:
vetor_bom = vetorizador.transform(["bom"])
print(vetor_bom.toarray())

[[1 0 0 0 0 0 0]]


In [10]:
vetor_um = vetorizador.transform(["um"])
print(vetor_um.toarray())

[[0 0 0 0 0 1 0]]


# Word2vec

In [53]:
from gensim.models import KeyedVectors
import numpy as np


In [12]:
!unzip "/content/cbow_s300.zip"

Archive:  /content/cbow_s300.zip
  inflating: cbow_s300.txt           


In [17]:
model = KeyedVectors.load_word2vec_format("/content/cbow_s300.txt")

In [19]:
len(model.get_vector('japao'))

300

In [31]:
model.most_similar('x-men')

[('vingadores', 0.6901994943618774),
 ('titãs', 0.5481435060501099),
 ('vilões', 0.5269604325294495),
 ('wolverine', 0.5259417295455933),
 ('super-heróis', 0.524213433265686),
 ('thunderbolts', 0.5063843131065369),
 ('heróis', 0.5006411671638489),
 ('mutantes', 0.4931108057498932),
 ('zumbis', 0.47079893946647644),
 ('mogadorianos', 0.4705388844013214)]

In [35]:
model.most_similar(positive=["dc","marvel"])

[('timely', 0.7087664604187012),
 ('nedor', 0.6748367547988892),
 ('panini', 0.571498692035675),
 ('wildstorm', 0.5172005891799927),
 ('mlj', 0.5102024078369141),
 ('crossgen', 0.5045009851455688),
 ('fawcett', 0.5008060932159424),
 ('archie', 0.49644556641578674),
 ('aircel', 0.4961481988430023),
 ('detective', 0.4733356833457947)]

In [36]:
model.most_similar(positive=["nuvens","estrela"], negative=["nuvem"])

[('estrelas', 0.5497430562973022),
 ('plêiades', 0.3791979253292084),
 ('colinas', 0.3746805489063263),
 ('trovoadas', 0.373703271150589),
 ('sombras', 0.37341946363449097),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727400779724),
 ('cigarras', 0.36065396666526794),
 ('galáxias', 0.35754913091659546),
 ('luas', 0.3575345277786255)]

In [44]:
model.most_similar(positive=["medico","mulher"], negative=["homem"])

[('enfermeira', 0.5567789077758789),
 ('esposa', 0.540894627571106),
 ('mãe', 0.5147395730018616),
 ('filha', 0.5052310824394226),
 ('namorada', 0.4970565736293793),
 ('filhinha', 0.49672731757164),
 ('irmã', 0.49583038687705994),
 ('parenta', 0.4849744439125061),
 ('sobrinha', 0.4843062162399292),
 ('babá', 0.48186033964157104)]

## Vetorização de texto

In [51]:
import nltk
import string
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [46]:
# Analise da estrutura do texto
artigo_treino.title.loc[12]

"Daniel Craig será stormtrooper em novo 'Star Wars', diz ator"

In [49]:
def tokenizador(texto):
  texto_minusculo = texto.lower()
  lista_alfanumerico = []
  for token_valido in nltk.word_tokenize(texto_minusculo):
    if token_valido in string.punctuation: continue
    lista_alfanumerico.append(token_valido)
  return lista_alfanumerico

In [52]:
tokenizador(artigo_treino.title.loc[12])

['daniel',
 'craig',
 'será',
 'stormtrooper',
 'em',
 'novo',
 "'star",
 'wars',
 'diz',
 'ator']

In [67]:
def combinacao_vetores_por_soma(palavras):
  vetor_resultante = np.zeros(300)
  for token in palavras:
    try:
      vetor_resultante += model.get_vector(token)
    except KeyError:
      if token.isnumeric():
        token = "0"*len(token)
        vetor_resultante += model.get_vector(token)
      else:
        vetor_resultante += model.get_vector("unknow")

  return vetor_resultante

In [71]:
palavras = tokenizador(artigo_treino.title.loc[12])
vetor_resultante = combinacao_vetores_por_soma(palavras)
print(len(vetor_resultante))
print(vetor_resultante)

300
[ 1.409091   -0.14778899 -0.15712103 -0.22756499  0.33947702  0.04513202
  0.10606999 -0.02744501  0.34897399 -0.622258   -1.18768101  0.07594701
  0.734674    0.605006   -1.41548402  0.231505   -0.93393402 -0.044746
 -1.116989   -0.01408298 -2.22429901 -0.98166602 -0.77250701  0.607256
  0.63582001  0.184902   -0.65532703  0.45974298  0.819684    0.61843801
  1.26487099 -0.23633801  0.39783901  0.68037201 -0.58962901  0.164664
  0.755284   -0.30015901 -0.85412398  0.60202201 -1.16588299  0.55755499
  1.20059     0.41932302  0.069055    1.11085396  0.07674501 -0.750762
  1.22592399 -0.51450599  0.05619898 -0.45865     1.11803401  0.560228
 -0.69264403 -0.32368899 -0.004358    0.53267698  0.03769496  0.164821
 -0.87106599 -0.273875    0.43975404  0.11239799 -0.48971699  0.38454201
  0.66049697  0.53603198 -0.70654999 -0.31440599 -0.45960598  0.06693399
 -0.196609   -0.23975599  0.384431   -0.69159699 -0.75968804 -0.58382899
 -1.405958   -0.19831699 -0.50192397  0.055928   -0.428562 